In [1]:
import pandas as pd
import jieba
from collections import Counter

scholar_data = pd.read_excel('学者撰写的论文摘要.xlsx')
ai_data = pd.read_excel('AI生成的论文摘要.xlsx')

scholar_abstracts = scholar_data['论文摘要']
ai_abstracts = ai_data['论文摘要']

# 加载停用词表
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        stopwords = set(file.read().strip().split('\n'))
    return stopwords

stopwords = load_stopwords('stopwords.txt')

# 分词并去除停用词
def jieba_tokenizer(text):
    words = jieba.cut(text)
    return ' '.join([word for word in words if word not in stopwords])

scholar_abstracts = scholar_abstracts.apply(jieba_tokenizer)
ai_abstracts = ai_abstracts.apply(jieba_tokenizer)

# 统计词频函数
def count_words(texts):
    all_words = []
    for text in texts:
        all_words.extend(text.split())
    return Counter(all_words)

scholar_words = count_words(scholar_abstracts)
ai_words = count_words(ai_abstracts)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\60938\AppData\Local\Temp\jieba.cache
Loading model cost 0.448 seconds.
Prefix dict has been built successfully.


In [2]:
# 输出词频最高的前15个词
print('学者撰写的论文摘要:')
print(scholar_words.most_common(15))
print('AI生成的论文摘要:')
print(ai_words.most_common(15))

学者撰写的论文摘要:
[(',', 747), ('学习', 218), ('方法', 170), ('深度', 159), ('进行', 149), ('算法', 139), ('目标', 137), ('研究', 137), ('检测', 134), ('中', 120), ('模型', 102), ('领域', 95), ('数据', 89), ('技术', 86), ('问题', 83)]
AI生成的论文摘要:
[('本文', 244), ('方法', 168), ('应用', 156), ('学习', 152), ('中', 150), ('深度', 115), ('研究', 107), ('领域', 88), ('算法', 84), ('模型', 83), ('检测', 82), ('技术', 78), ('（', 76), ('）', 76), ('方向', 73)]


In [3]:
# 统计独有词
scholar_unique_words = set(scholar_words.keys()) - set(ai_words.keys())
ai_unique_words = set(ai_words.keys()) - set(scholar_words.keys())

# 分别输出独有词频数最高的前15个词
print('学者撰写的论文摘要独有词:')
print(Counter({word: scholar_words[word] for word in scholar_unique_words}).most_common(15))
print('AI生成的论文摘要独有词:')
print(Counter({word: ai_words[word] for word in ai_unique_words}).most_common(15))

学者撰写的论文摘要独有词:
[(';', 70), (')', 70), ('(', 67), ('代表性', 16), ('文中', 15), ('归纳', 15), ('目前', 13), ('分为', 12), ('角度', 11), ('值', 10), ('算子', 10), ('"', 10), ('所提', 9), ('节点', 9), ('科学', 9)]
AI生成的论文摘要独有词:
[('揭示', 16), ('涵盖', 8), ('系列', 7), ('FCN', 7), ('最新进展', 7), ('表现出色', 6), ('交通系统', 6), ('系统阐述', 5), ('如多', 4), ('层次化', 4), ('DQN', 4), ('高精度', 4), ('局限', 4), ('优异', 4), ('现代', 4)]


In [4]:
import nltk
from nltk import ngrams

# 定义函数来生成 N-gram，并计算频率
def generate_ngrams(tokens, n):
    n_grams = ngrams(tokens, n)
    return [' '.join(gram) for gram in n_grams]

# 定义函数来统计 N-gram 的频率并返回前 top_n 个
def top_ngrams_by_frequency(abstracts, min_n, max_n, top_n):
    all_ngrams = Counter()
    for abstract in abstracts:
        words = abstract.split()
        for n in range(min_n, max_n + 1):
            ngrams_list = generate_ngrams(words, n)  # 生成 N-gram
            all_ngrams.update(ngrams_list)
    top_ngrams = all_ngrams.most_common(top_n)
    return top_ngrams

In [ ]:
# 定义输出 top_n 个高频搭配的函数
def print_top_ngrams(top_ngrams, category):
    print(f"Top {len(top_ngrams)} {category} 搭配：")
    for ngram, freq in top_ngrams:
        print(f"{ngram}: {freq}")

In [6]:
# 统计学者撰写的摘要中的 N-gram 频率并输出前15个
scholar_top_ngrams = top_ngrams_by_frequency(scholar_abstracts, 2, 10, 30)
print_top_ngrams(scholar_top_ngrams, "学者撰写")

Top 30 学者撰写 搭配：
深度 学习: 106
目标 检测: 92
强化 学习: 50
计算机 视觉: 38
问题 ,: 33
检测 算法: 28
目标 检测 算法: 27
方法 ,: 27
卷积 神经网络: 26
智能 体: 26
, 提出: 25
机器 学习: 24
, 深度: 22
; 最后: 20
提出 一种: 20
路径 规划: 19
检测 方法: 19
研究 方向: 18
, 介绍: 18
进行 分析: 18
发展 方向: 17
数据 集: 17
最后 ,: 17
进行 展望: 16
进行 总结: 16
语义 分割: 16
实验 结果表明: 16
, 提出 一种: 16
, 深度 学习: 15
, 分析: 15


In [7]:
# 统计AI生成的摘要中的 N-gram 频率并输出前15个
ai_top_ngrams = top_ngrams_by_frequency(ai_abstracts, 2, 10, 30)
print_top_ngrams(ai_top_ngrams, "AI生成")

Top 30 AI生成 搭配：
深度 学习: 91
目标 检测: 55
研究 方向: 42
本文 综述: 41
实际 应用: 35
应用 中: 35
中 应用: 35
强化 学习: 33
未来 研究: 30
最后 本文: 30
未来 研究 方向: 29
实际 应用 中: 29
卷积 神经网络: 28
本文 提出: 28
计算机 视觉: 26
神经网络 （: 26
未来 发展: 25
发展 方向: 24
提出 一种: 24
本文 提出 一种: 24
本文 系统: 23
未来 发展 方向: 23
本文 详细: 21
本文 探讨: 20
本文 总结: 20
检测 算法: 19
目标 检测 算法: 18
卷积 神经网络 （: 18
详细 讨论: 18
实验 结果表明: 18
